# Chapter 10 - Logging Patterns

## What is logging?

- Cross cutting concern (Applies to the entire application)
- Reasons
    * Error diagnoisis
    * Debugging
    * Performance monitoring
    * Security auditing
    * Compliance
    * Behavioral analytics
- Log levels
    Enum: `Microsoft.Extensions.Logging.LogLevel`
- Structure
    * A single string (Unstructured)
    * Structrured i.e. Json

## Writing logs

- Provider based - Register 1 or more `ILoggerProvider` instances
- Logger interfaces: ` ILogger, ILogger<T>, or ILoggerFactory`
- Logged to console by default

    ```csharp
    namespace LoggingConsole;
    public class ServiceUsingILogger(ILogger<ServiceUsingILogger> logger)
    {
        private readonly ILogger _logger = logger ?? throw new ArgumentNullException(nameof(logger));

        public string Generate()
        {
            _logger.LogInformation("ServiceUsingILogger generating a GUID...");
            var guid = Guid.NewGuid();
            _logger.LogDebug("ServiceUsingILogger generated the GUID {guid}.", 
            guid);
            return guid.ToString();
        }
    }
    ```

- Using `ILoggerFactory.CreateLogger()` allows us to pass a category name
- Recommends using `ILogger<T>` by default

## Log levels

| Level | Method    | Description                                           | 
| ------|-----------|-------------------------------------------------------|
| Trace | LogTrace  | Capture detailed info, performance, sensitive info?   |
| Debug | LogDebug  | Log debug info and development info                   |
| Info  | LogInformation | Use to track flow, normal events                 |
| Warning | LogWarning | Log abnormal behavior                              |
| Error | LogError  | Log errors                                            |
| Critical | LogCritical | Log errors that require imediate attention, Stop state | 

- Better performance to use **log message templates** rather than string interpolation for logging

    ```csharp
    _logger.LogTrace("Some: {variable}", variable);
    // Or
    _logger.LogTrace("Some: {0}", variable);
    ```
    Delays the processing until it is sure it must log the message

    It may be ok to use string interpolation if logging info or a level is likley to be logged regardless of environment i.e. Warning+ 
    Better to keep it conistent

## Logging Providers

- Console (Default)
- Debug: Debugger monitor
- EventSource: Built-in provider for structured logging
- EventLog: Windows event log
- ApplicationInsights: Integrates with Azure Application INsights

Many other third party providers i.e. Serilog, Log4Net

## Configuring

appsettings.json

```json
{
 "Logging": {
 "LogLevel": {
 "Default": "Information",
 "Microsoft": "Warning"
 }
 }
}
```

appsettings.development.json

```json
{
    "Logging": {
        "LogLevel": {
            "Default": "Information",
            "Microsoft": "Warning"
        },
        "Console": {
            "LogLevel": {
                "Default": "Trace"
            }
        }
    }
}
```

Can use filters in code.

```csharp
builder.Logging.AddFilter<ConsoleLoggerProvider>(
    ServiceUsingLoggerFactory.CategoryName,
    level => level >= LogLevel.Debug
);
```

## Structured logging

Built-in Json formatter

```csharp
builder.Logging.AddJsonConsole();
...
logger.LogInformation("You hit the {category} URL!", category);
```

```json
{
 "EventId": 0,
 "LogLevel": "Information",
 "Category": "root",
 "Message": "You hit the root URL!",
 "State": {
 "Message": "You hit the root URL!",
 "category": "root",
 "{OriginalFormat}": "You hit the {category} URL!"
 }
}
```

Some third part providers such as `Serilog` have more capabilities.

See https://github.com/serilog/serilog-aspnetcore/blob/dev/samples/Sample/Program.cs for an example.

In [1]:
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.Extensions.Hosting.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Mvc.ViewFeatures.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Diagnostics.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.Results.dll"

In [2]:
string[] args = {"--urls","http://localhost:7002","--appsettings","C10.appsettings.json"};

In [ ]:
using Microsoft.AspNetCore.Http;
using Microsoft.AspNetCore.Http.HttpResults;
using Microsoft.AspNetCore.Routing;
using Microsoft.AspNetCore.Builder;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using System.IO;

//This section is not part of the example, just a way to redirect the logs to a file for viewing since I cannot see the console in the notebook
StreamWriter sw = File.AppendText("logs/applog.txt");
sw.AutoFlush = true;
Console.SetError(sw);
Console.SetOut(sw);

var builder = WebApplication.CreateBuilder(args);
builder.Logging.AddJsonConsole();

var app = builder.Build();

app.MapGet("/", (ILoggerFactory loggerFactory) =>
{
    const string category = "root";
    var logger = loggerFactory.CreateLogger(category);
    logger.LogInformation("You hit the {category} URL!", category);
    return "Hello World!";
});

app.RunAsync();


In [7]:
app.StopAsync();

In [5]:
using System.Net.Http;

var httpClient = new HttpClient();

var response = await httpClient.GetAsync("http://localhost:7002/");
response.EnsureSuccessStatusCode();
var content = await response.Content.ReadAsStringAsync();
content.DisplayAs("application/json")

## OpenTelemetry example

Example using the OpenTelemetry log provider

- https://github.com/open-telemetry/opentelemetry-dotnet/tree/main/docs/logs/getting-started-aspnetcore
- https://opentelemetry.io/ecosystem/registry/?component=exporter&language=dotnet
- https://github.com/DataDog/dd-trace-dotnet/blob/master/tracer/samples/OpenTelemetry

In [2]:
#r "nuget:OpenTelemetry.Exporter.Console"
#r "nuget:OpenTelemetry.Extensions.Hosting"

Installed Packages OpenTelemetry.Exporter.Console, 1.12.0 OpenTelemetry.Extensions.Hosting, 1.12.0

In [3]:
string[] args = {"--urls","http://localhost:7003","--appsettings","C10.appsettings-OT.json"};

In [10]:

using Microsoft.AspNetCore.Http;
using Microsoft.AspNetCore.Http.HttpResults;
using Microsoft.AspNetCore.Routing;
using Microsoft.AspNetCore.Builder;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using OpenTelemetry.Logs;
using OpenTelemetry.Resources;
using System.IO;

//This section is not part of the example, just a way to redirect the logs to a file for viewing since I cannot see the console in the notebook
StreamWriter sw = File.AppendText("logs/OT-console.txt");
sw.AutoFlush = true;
Console.SetError(sw);
Console.SetOut(sw);

var builder = WebApplication.CreateBuilder(args);

// For instructional purposes only, disable the default .NET logging providers.
// We remove the console logging provider in this demo to use the verbose
// OpenTelemetry console exporter instead. For most development and production
// scenarios the default console provider works well and there is no need to
// clear these providers.
builder.Logging.ClearProviders();

// Add OpenTelemetry logging provider by calling the WithLogging extension.
builder.Services.AddOpenTelemetry()
    .ConfigureResource(r => r.AddService(builder.Environment.ApplicationName))
    .WithLogging(logging => logging
        /* Note: ConsoleExporter is used for demo purpose only. In production
           environment, ConsoleExporter should be replaced with other exporters
           (e.g. OTLP Exporter). */
        .AddConsoleExporter());

var app = builder.Build();

app.MapGet("/", (ILogger<Program> logger) =>
{
    var greeting = "Hello, World!";
    logger.LogInformation("Greeting: {greeting} ", greeting);
    return "Hello World!";
});

app.Logger.LogInformation("App starting...");

app.RunAsync();

public partial class Program { }



In [14]:
app.StopAsync();

In [13]:
using System.Net.Http;

var httpClient = new HttpClient();

var response = await httpClient.GetAsync("http://localhost:7003/");
response.EnsureSuccessStatusCode();
var content = await response.Content.ReadAsStringAsync();
content.DisplayAs("application/json")